# 🛵 Análise Temporal de Sinistros - Heatmap Interativo (Infosiga 2022–2025)



Este notebook realiza uma análise espacial e temporal de sinistros no estado de São Paulo. Os dados são provenientes do portal Infosiga SP e incluem informações detalhadas sobre a ocorrência dos sinistros e os tipos de veículos envolvidos.

As etapas contemplam:
- Leitura e tratamento dos dados brutos em formato CSV;
- Filtragem espacial dos sinistros localizados dentro dos limites do estado de São Paulo;
- Junção com a base de veículos para poder filtrar por típo de veículo;
- Geração de um **mapa de calor temporal animado** utilizando Folium e GeoPandas.

O objetivo é facilitar a visualização da distribuição espaço-temporal desses eventos, contribuindo para análises de segurança viária e planejamento urbano.


## 📄 Documentação Técnica

📁 **Arquivos Utilizados**

- `sinistros_2022-2025.csv`: base de dados de sinistros do Infosiga, com coordenadas, tipo de sinistro e data da ocorrência;
- `veiculos_2022-2025.csv`: base de veículos envolvidos nos sinistros, com campo de tipo do veículo;
- `sp_uf_2023.gpkg`: arquivo vetorial com o limite geográfico do estado de São Paulo (CRS: EPSG:4674).

**🔧 Etapas do Processo**

1. **Leitura e preparação dos dados**:
   - Conversão de coordenadas para `float` e correção de sinais;
   - Geração de pontos geográficos (GeoDataFrame);
   - Filtragem espacial para manter apenas sinistros ocorridos dentro do estado de SP.

2. **Filtragem específica**:
   - Seleção por tipo de registro (`SINISTRO FATAL`);
   - Foco no município de Campinas;
   - Junção com a base de veículos para isolar os casos com `tipo_veiculo = MOTOCICLETA`.

3. **Tratamento temporal e visualização**:
   - Conversão da data do sinistro para o tipo `datetime`;
   - Agrupamento mensal das ocorrências;
   - Geração de mapa de calor animado com `HeatMapWithTime` (Folium).

🧩 **Personalizações possíveis**

- Alterar o município ou período analisado no bloco de filtros;
- Trocar o tipo de veículo ou incluir múltiplos;
- Adaptar para outras categorias de acidentes (`tipo_acidente_primario`);
- Usar um gradiente de cor personalizado para o mapa de calor.

**💡 Requisitos**

- Python 3.9+ no ambiente do Google Colab
- Bibliotecas: `pandas`, `geopandas`, `folium`, `shapely`

---

Em caso de dúvidas ou para replicar com outros municípios ou tipos de veículos, basta ajustar os filtros nos blocos correspondentes.


# ✅ Análise Tempora de Sinistros - Mapa Temporal com Folium

In [25]:

# Instalar bibliotecas (se necessário)
!pip install geopandas folium


##Filtros

In [26]:
# 🔧 Parâmetros de análise

# Filtros principais
municipio_alvo = 'CAMPINAS'
tipo_registro_alvo = 'SINISTRO FATAL'
tipo_veiculo_alvo = 'MOTOCICLETA'

# Ativar ou não filtros opcionais
filtrar_por_tipo_via = False
tipo_via_alvo = 'VIAS MUNICIPAIS'

filtrar_por_mes = False
meses_alvo = ['2024/01', '2024/02', '2024/03']

filtrar_por_tipo_acidente = False
tipo_acidente_alvo = 'ATROPELAMENTO'


##Processamento

### 1. Montar o Google Drive

In [27]:

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 2. Importar bibliotecas necessárias

In [28]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import folium
from folium.plugins import TimestampedGeoJson
from folium.plugins import HeatMapWithTime

### 3. Ler os dados de sinistros e veículos

In [29]:

# Ajuste os caminhos conforme sua estrutura no Drive
caminho_csv_sinistros = '/content/drive/MyDrive/Analises_DETRAN/infosiga-temporal/sinistros_2022-2025.csv'
caminho_csv_veiculos = '/content/drive/MyDrive/Analises_DETRAN/infosiga-temporal/veiculos_2022-2025.csv'
caminho_sp = 'https://github.com/carolcattaneo/infosiga-temporal/raw/refs/heads/main/SP_UF_2023.gpkg'

# Lê os dados de sinistros
df = pd.read_csv(caminho_csv_sinistros, sep=';', encoding='latin1')
df['lat'] = pd.to_numeric(df['lat'], errors='coerce')
df['long'] = pd.to_numeric(df['long'], errors='coerce')
df['lat'] = df['lat'].apply(lambda x: -abs(x) if x > 0 else x)
df['long'] = df['long'].apply(lambda x: -abs(x) if x > 0 else x)
df = df.dropna(subset=['lat', 'long'])
geometry = [Point(xy) for xy in zip(df['long'], df['lat'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326')

# Filtra pelo estado de São Paulo
estado_sp = gpd.read_file(caminho_sp)
estado_sp = estado_sp.to_crs(gdf.crs)
gdf = gdf[gdf.within(estado_sp.unary_union)]


<ipython-input-29-75622110284b>:19: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  gdf = gdf[gdf.within(estado_sp.unary_union)]


### 4. Aplicar filtros iniciais nos sinistros

In [30]:
# Explora os valores únicos de colunas-chave
print("Valores únicos de 'tipo_registro':")
print(gdf['tipo_registro'].unique())

print("\nValores únicos de 'ano_mes_sinistro':")
print(gdf['ano_mes_sinistro'].unique())

print("\nValores únicos de 'tipo_via':")
print(gdf['tipo_via'].unique())

print("\nValores únicos de 'tipo_acidente_primario':")
print(gdf['tipo_acidente_primario'].unique())

Valores únicos de 'tipo_registro':
['NOTIFICACAO' 'SINISTRO FATAL' 'SINISTRO NAO FATAL']

Valores únicos de 'ano_mes_sinistro':
['2022/01' '2022/02' '2022/03' '2022/04' '2022/05' '2022/06' '2022/07'
 '2022/08' '2022/09' '2022/10' '2022/11' '2022/12' '2023/01' '2023/02'
 '2023/03' '2023/04' '2023/05' '2023/06' '2023/07' '2023/08' '2023/09'
 '2023/10' '2023/11' '2023/12' '2024/01' '2024/02' '2024/03' '2024/04'
 '2024/05' '2024/06' '2024/07' '2024/08' '2024/09' '2024/10' '2024/11'
 '2024/12' '2025/01' '2025/02' '2025/03']

Valores únicos de 'tipo_via':
['VIAS MUNICIPAIS' 'RODOVIAS' 'NAO DISPONIVEL']

Valores únicos de 'tipo_acidente_primario':
['OUTROS' 'CHOQUE' 'ATROPELAMENTO' 'COLISAO' 'NAO DISPONIVEL']


In [31]:
# Filtros no gdf (sinistros)

filtros = (
    (gdf['tipo_registro'] == tipo_registro_alvo) &
    (gdf['municipio'].str.upper() == municipio_alvo.upper())
)

if filtrar_por_mes:
    filtros &= gdf['ano_mes_sinistro'].isin(meses_alvo)

if filtrar_por_tipo_via:
    filtros &= gdf['tipo_via'] == tipo_via_alvo

if filtrar_por_tipo_acidente:
    filtros &= gdf['tipo_acidente_primario'] == tipo_acidente_alvo

gdf_filtrado = gdf[filtros]

### 5. Cruzar com dados de veículos e filtrar por tipo de veículo

In [32]:

veiculos_df = pd.read_csv(caminho_csv_veiculos, sep=';', encoding='latin1')
gdf_veiculos = gdf_filtrado.merge(veiculos_df[['id_sinistro', 'tipo_veiculo']], on='id_sinistro', how='inner')
gdf_tipo_veiculo = gdf_veiculos[gdf_veiculos['tipo_veiculo'].str.upper() == tipo_veiculo_alvo.upper()]
# Tipos disponíveis: 'AUTOMOVEL', 'BICICLETA', 'CAMINHAO', 'MOTOCICLETA', 'NAO DISPONIVEL', 'ONIBUS', 'OUTROS'

### 6. Preparar campo de data para animação

In [35]:

gdf_tipo_veiculo['data_sinistro'] = pd.to_datetime(gdf_tipo_veiculo['data_sinistro'], errors='coerce')


<ipython-input-35-62ae11d5702c>:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  gdf_tipo_veiculo['data_sinistro'] = pd.to_datetime(gdf_tipo_veiculo['data_sinistro'], errors='coerce')
/usr/local/lib/python3.11/dist-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


### 7. Criar GeoJSON temporal para o mapa

In [37]:

features = []
for _, row in gdf_tipo_veiculo.iterrows():
    if pd.notnull(row['data_sinistro']):
        features.append({
            'type': 'Feature',
            'geometry': row['geometry'].__geo_interface__,
            'properties': {
                'time': row['data_sinistro'].isoformat(),
                'popup': f"Data: {row['data_sinistro'].date()}<br>Tipo: {row['tipo_acidente_primario']}"
            }
        })

geojson = {
    'type': 'FeatureCollection',
    'features': features
}


## Criar o mapa interativo com controle temporal

In [38]:
from folium.plugins import HeatMapWithTime
import folium

# 8.1 Garante que a data está no formato correto
gdf = gdf_tipo_veiculo.dropna(subset=['data_sinistro', 'geometry']).copy()
gdf['ano_mes'] = gdf['data_sinistro'].dt.to_period('M')

# 8.2 Ordena os períodos únicos
periodos = sorted(gdf['ano_mes'].unique())

# 8.3 Agrupa as coordenadas por mês
heat_data = []
for periodo in periodos:
    grupo = gdf[gdf['ano_mes'] == periodo]
    coords = [[row.geometry.y, row.geometry.x] for row in grupo.itertuples() if row.geometry is not None]
    heat_data.append(coords)

# 8.4 Cria o mapa centralizado no centroide dos dados
centro = gdf.geometry.unary_union.centroid
m = folium.Map(location=[centro.y, centro.x], zoom_start=12)

# 8.5 Adiciona o mapa de calor com controle temporal
HeatMapWithTime(
    data=heat_data,
    index=[str(p) for p in periodos],
    auto_play=False,
    max_opacity=0.65,
    radius=20,
    use_local_extrema=True,
    # gradient=viridis_gradient  # descomente se quiser usar gradiente personalizado
).add_to(m)

# 8.6 Salva como HTML
m.save('heatmap_temporal_mensal.html')

# Para exibir diretamente no notebook (opcional)
m


<ipython-input-38-1c7e4458a640>:19: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  centro = gdf.geometry.unary_union.centroid


**🗺️ Como usar o mapa interativo**

Este mapa apresenta a **distribuição espaço-temporal de sinistros envolvendo motocicletas** no município de Campinas, com base nos dados do Infosiga.

**🔧 Funcionalidades principais:**

* **Controle temporal (barra inferior)**:
  Permite navegar entre os meses do período analisado (2022–2025). Use os botões ▶️⏸️ para iniciar ou pausar a animação, ou arraste manualmente o controle para visualizar um mês específico.

* **Zoom e movimentação**:
  Use o scroll do mouse ou os botões de `+` e `–` no canto superior esquerdo para dar zoom. Você também pode clicar e arrastar o mapa para explorar diferentes regiões da cidade.

* **Densidade por cores**:
  As cores mais **intensas (vermelho/roxo)** indicam maior concentração de sinistros naquele local e mês. Cores mais **leves (amarelo/verde)** indicam menor concentração.

> 💡 Dica: ao visualizar diferentes meses, observe padrões de concentração em determinadas vias ou bairros. Isso pode indicar recorrência de riscos em pontos específicos ao longo do tempo.

---